# Mnist classification demo

In [9]:
%%capture
import grpc 
import numpy as np
import hydro_serving_grpc as hs
import keras
from keras.datasets import mnist
from keras import backend as K
import os

## Read data sample

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Setup channel

In [11]:
channel = grpc.insecure_channel("localhost:9090")
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="mnist_tf")

## Create tensors

In [29]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=-1), hs.TensorShapeProto.Dim(size=784)])
tensor = hs.TensorProto(dtype=hs.DT_FLOAT, tensor_shape=tensor_shape, float_val=np.array([x_test[0]]).flatten().astype(np.float32))

## Send request to model servable

In [31]:
float_val=np.array([x_test[0]]).flatten().astype(np.float32)
request = hs.PredictRequest(model_spec=model_spec, inputs={"imgs": tensor}) 
result = stub.Predict(request)
print('Output value:')
result= np.array(result.outputs['probabilities'].float_val)
print(np.array(result).argmax())
print('Ground truth label:')
print(y_test[0].argmax())

Output value:
7
Ground truth label:
7
